In [1]:
import os
import json
import itertools

## TabMWP - Chameleon GPT-4

In [2]:
result_file = "../../results/tabmwp/chameleon_gpt4_test_cache.jsonl"
data_file = "../../data/tabmwp/problems_test.json"

prob_data = json.load(open(data_file))

with open(result_file) as f:
    results = {}
    for line in f:
        try:
            data = json.loads(line)
            results[data["pid"]] = data
        except:
            pass

In [3]:
modules_list = []

for pid, result in results.items():
    modules = result["modules:output"]
    if "program_generator_and_verifier" in modules:
        index = modules.index("program_generator_and_verifier")
        nested = []
        for module in modules:
            if module == "program_generator_and_verifier":
                nested.append(["program_generator", "program_verifier"])
            else:
                nested.append([module])
        modules = [m for sublist in nested for m in sublist]
        #print(modules)
    
    modules = ["START"] + modules + ["END"]
    modules_list.append(modules)

In [4]:
# modules_list

In [5]:
transition_num = {
 "START": {},
'program_generator': {},
 'program_verifier': {},
 'program_executor': {},
 'answer_generator': {},
 'row_lookup': {},
 'column_lookup': {},
 'table_verbalizer': {},
 'knowledge_retrieval': {},
 'solution_generator': {}
}

transitions = {
 "START": {},
'program_generator': {},
 'program_verifier': {},
 'program_executor': {},
 'answer_generator': {},
 'row_lookup': {},
 'column_lookup': {},
 'table_verbalizer': {},
 'knowledge_retrieval': {},
 'solution_generator': {}
}

In [6]:
for modules in modules_list:
    for i, module in enumerate(modules[:-1]):
        next_module = modules[i+1]
        if next_module not in transition_num[module]:
            transition_num[module][next_module] = 1
        else:
            transition_num[module][next_module] += 1
            
        transitions[module][next_module] = round(transition_num[module][next_module] / sum(transition_num[module].values()), 2)

In [7]:
transition_num

{'START': {'program_generator': 2996,
  'row_lookup': 2032,
  'knowledge_retrieval': 1443,
  'solution_generator': 810,
  'column_lookup': 405},
 'program_generator': {'program_verifier': 5533},
 'program_verifier': {'program_executor': 5533},
 'program_executor': {'answer_generator': 5528, 'solution_generator': 5},
 'answer_generator': {'END': 7686},
 'row_lookup': {'table_verbalizer': 541,
  'knowledge_retrieval': 494,
  'solution_generator': 535,
  'column_lookup': 64,
  'program_generator': 400},
 'column_lookup': {'program_generator': 349,
  'solution_generator': 116,
  'row_lookup': 2,
  'knowledge_retrieval': 2},
 'table_verbalizer': {'program_generator': 571},
 'knowledge_retrieval': {'program_generator': 1216,
  'solution_generator': 693,
  'table_verbalizer': 30},
 'solution_generator': {'answer_generator': 2158, 'program_generator': 1}}

In [8]:
transitions

{'START': {'program_generator': 0.39,
  'row_lookup': 0.26,
  'knowledge_retrieval': 0.19,
  'solution_generator': 0.11,
  'column_lookup': 0.05},
 'program_generator': {'program_verifier': 1.0},
 'program_verifier': {'program_executor': 1.0},
 'program_executor': {'answer_generator': 1.0, 'solution_generator': 0.0},
 'answer_generator': {'END': 1.0},
 'row_lookup': {'table_verbalizer': 0.27,
  'knowledge_retrieval': 0.24,
  'solution_generator': 0.26,
  'column_lookup': 0.03,
  'program_generator': 0.2},
 'column_lookup': {'program_generator': 0.74,
  'solution_generator': 0.25,
  'row_lookup': 0.02,
  'knowledge_retrieval': 0.02},
 'table_verbalizer': {'program_generator': 1.0},
 'knowledge_retrieval': {'program_generator': 0.63,
  'solution_generator': 0.36,
  'table_verbalizer': 0.02},
 'solution_generator': {'answer_generator': 1.0, 'program_generator': 0.0}}

In [9]:
for module in transitions:
    print(module.replace("_", " "))
    
    next_module = transitions[module]
    for m in next_module:
        print(f" -> {m}: {next_module[m]}".replace("_", " "))
        
    print("")

START
 -> program generator: 0.39
 -> row lookup: 0.26
 -> knowledge retrieval: 0.19
 -> solution generator: 0.11
 -> column lookup: 0.05

program generator
 -> program verifier: 1.0

program verifier
 -> program executor: 1.0

program executor
 -> answer generator: 1.0
 -> solution generator: 0.0

answer generator
 -> END: 1.0

row lookup
 -> table verbalizer: 0.27
 -> knowledge retrieval: 0.24
 -> solution generator: 0.26
 -> column lookup: 0.03
 -> program generator: 0.2

column lookup
 -> program generator: 0.74
 -> solution generator: 0.25
 -> row lookup: 0.02
 -> knowledge retrieval: 0.02

table verbalizer
 -> program generator: 1.0

knowledge retrieval
 -> program generator: 0.63
 -> solution generator: 0.36
 -> table verbalizer: 0.02

solution generator
 -> answer generator: 1.0
 -> program generator: 0.0

